### Import Statements

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

## Google docs connection

In [ ]:
#Execute the cell and follow the steps, it is necessary to connect with the user account in which the files are contained.from google.colab import drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Scrapping Data Launches

In [ ]:
#Getting the Table’s Elements with BeautifulSoup
webpage = "https://nextspaceflight.com/launches/past/?page=1&search=" # Go to the record of past spaceflights.

spaceflights_list = []
while True:
  space_flights = requests.get(webpage)
  soup = BeautifulSoup(space_flights.content, "lxml")
  records = soup.find_all("div", {"class":"mdl-cell mdl-cell--6-col"})
  for cell in records:
    #Setting variables
    organisation = None
    detail = None
    date = None
    location = None
    mission_status = None
    price = None
    rocket_status = None

    organisation = cell.find("div", {"class":"mdl-card__title-text"}).text.strip() #Organisation
    detail = cell.find("h5", {"class":"header-style"}).text.strip() #Detail

    info = cell.find("div", {"class":"mdl-card__supporting-text"})#Date, Location
    date = info.text.strip().split('\n')[0]
    location = info.find('br').next_sibling.strip()

    #Scrape launch details
    details_button = cell.find("button", {"class":"mdc-button"}).get('onclick').split("=")[1].replace("'", '').strip() #replace will drop the ' character
    details_webpage = requests.get(f"https://nextspaceflight.com{details_button}")
    
    launch_details = BeautifulSoup(details_webpage.content, "lxml")

    mission_status = launch_details.find("h6", {"class":"rcorners status"}).text.strip() #Capture Mission Status.

    rocket_details = launch_details.find("div", {"class":"mdl-cell mdl-cell--6-col-desktop mdl-cell--4-col-tablet mdl-cell--4-col-phone"})

    #Enter the rocket table and get Rocket Status and Price.
    for rocket_aspect in rocket_details.find_all("div", {"class":"mdl-cell mdl-cell--6-col-desktop mdl-cell--12-col-tablet"}):

      detail_rocket = rocket_aspect.text.split(":")
      if detail_rocket[0] == "Price":
        price = detail_rocket[1]
      if detail_rocket[0] == "Status":
        rocket_status = detail_rocket[1]

    #Create dataframe row with launch data. We can add multiple values to list with extend()
    rocket = [organisation, location, date, detail, rocket_status, price, mission_status]

    #Adding Rocket Launch Data to  
    spaceflights_list.append(rocket)

  #Go to next page.
  right_div = soup.find("div", {"style":"float: right; margin-right: 5px"})
  if right_div: # If there is another page
    next_page = right_div.find("button", {"class":"mdc-button mdc-button--raised"}).get('onclick').split(" = ")[1].replace("'", '').strip() 
    webpage = f"https://nextspaceflight.com/launches/past/{next_page}"  

  else:
    break

#Converting Spaceflights List to Pandas Dataframe.
spaceflights_data = pd.DataFrame(spaceflights_list, columns=['Organisation', 'Location', 'Date',
                                              'Detail', 'Rocket_Status', 'Price', 'Mission_Status'])

In [ ]:
#Checking data scrapped.
spaceflights_data.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Sun Sep 11, 2022 01:20 UTC","Falcon 9 Block 5 | Starlink Group 4-2, BlueWal...",Active,$67.0 million,Success
1,Arianespace,"ELA-3, Guiana Space Centre, French Guiana, France","Wed Sep 07, 2022 21:45 UTC",Ariane 5 ECA | Eutelsat Konnect VHTS,Active,$200.0 million,Success
2,CASC,"LC-3, Xichang Satellite Launch Center, China","Tue Sep 06, 2022 04:19 UTC",Long March 2D | Yaogan 35 Group 05,Active,$29.75 million,Success
3,CASIC,"Site 95, Jiuquan Satellite Launch Center, China","Tue Sep 06, 2022 02:24 UTC",Kuaizhou 1A | CentiSpace-1 S3 & S4,Active,$5.8 million,Success
4,SpaceX,"SLC-40, Cape Canaveral SFS, Florida, USA","Mon Sep 05, 2022 02:09 UTC",Falcon 9 Block 5 | Starlink Group 4-20 and Var...,Active,$67.0 million,Success


In [ ]:
# Examine the data types
spaceflights_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6378 entries, 0 to 6377
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Organisation    6378 non-null   object
 1   Location        6378 non-null   object
 2   Date            6378 non-null   object
 3   Detail          6378 non-null   object
 4   Rocket_Status   6378 non-null   object
 5   Price           2376 non-null   object
 6   Mission_Status  6378 non-null   object
dtypes: object(7)
memory usage: 348.9+ KB


# Storing Data Launches

In [ ]:
#Transform and store data to a comma-separated values (csv) file.
path = "Your_path_to_save"
spaceflights_data.to_csv(path_or_buf=f"{path}spaceflights_data.csv")